In [1]:
from pathlib import Path
import os
from collections import deque

In [2]:
fp = os.path.join(Path().absolute(), "inputs", "input18.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input18_test.txt")

with open(fp, "r") as f:
    data = f.read().split("\n")[:-1]

In [3]:
data

['R 5 (#4f4602)',
 'U 8 (#2c84c3)',
 'R 7 (#5ad612)',
 'D 3 (#4fe053)',
 'R 3 (#469872)',
 'D 8 (#3089f3)',
 'R 2 (#74f592)',
 'D 7 (#716733)',
 'R 4 (#666e72)',
 'U 5 (#4d3843)',
 'R 7 (#6e9692)',
 'U 5 (#721373)',
 'R 4 (#0e6cd2)',
 'U 8 (#1e2543)',
 'R 7 (#464822)',
 'U 8 (#1984a3)',
 'R 2 (#523d02)',
 'U 3 (#1984a1)',
 'L 9 (#10c6c2)',
 'U 5 (#1e2541)',
 'L 6 (#449642)',
 'U 6 (#504713)',
 'L 4 (#00fdb2)',
 'U 3 (#383883)',
 'L 3 (#470ca0)',
 'U 9 (#6653a3)',
 'L 4 (#1941c0)',
 'U 7 (#031ba3)',
 'L 4 (#9cfe40)',
 'D 4 (#389153)',
 'L 11 (#010280)',
 'D 2 (#7139e3)',
 'R 11 (#15d4a0)',
 'D 5 (#935731)',
 'L 5 (#448f80)',
 'D 5 (#20f881)',
 'L 6 (#8aa820)',
 'U 4 (#3bb9e1)',
 'L 6 (#6e2700)',
 'U 3 (#4113e3)',
 'L 4 (#18af40)',
 'U 2 (#38add3)',
 'L 6 (#5369b0)',
 'U 7 (#7647e3)',
 'L 4 (#610f40)',
 'U 5 (#50e583)',
 'L 6 (#2294f0)',
 'U 6 (#59f503)',
 'L 3 (#0e3af0)',
 'U 3 (#5cde83)',
 'L 7 (#0e3af2)',
 'U 7 (#618133)',
 'L 8 (#14fbf2)',
 'U 4 (#1ab3b3)',
 'L 3 (#6d6092)',
 'U 10 (

# Part 1

In [4]:
data_processed = []
for line in data:
    direction, num, colour = line.split()
    num = int(num)
    colour = colour[1:-1]
    data_processed.append((direction, num, colour))

In [5]:
data_processed

[('R', 5, '#4f4602'),
 ('U', 8, '#2c84c3'),
 ('R', 7, '#5ad612'),
 ('D', 3, '#4fe053'),
 ('R', 3, '#469872'),
 ('D', 8, '#3089f3'),
 ('R', 2, '#74f592'),
 ('D', 7, '#716733'),
 ('R', 4, '#666e72'),
 ('U', 5, '#4d3843'),
 ('R', 7, '#6e9692'),
 ('U', 5, '#721373'),
 ('R', 4, '#0e6cd2'),
 ('U', 8, '#1e2543'),
 ('R', 7, '#464822'),
 ('U', 8, '#1984a3'),
 ('R', 2, '#523d02'),
 ('U', 3, '#1984a1'),
 ('L', 9, '#10c6c2'),
 ('U', 5, '#1e2541'),
 ('L', 6, '#449642'),
 ('U', 6, '#504713'),
 ('L', 4, '#00fdb2'),
 ('U', 3, '#383883'),
 ('L', 3, '#470ca0'),
 ('U', 9, '#6653a3'),
 ('L', 4, '#1941c0'),
 ('U', 7, '#031ba3'),
 ('L', 4, '#9cfe40'),
 ('D', 4, '#389153'),
 ('L', 11, '#010280'),
 ('D', 2, '#7139e3'),
 ('R', 11, '#15d4a0'),
 ('D', 5, '#935731'),
 ('L', 5, '#448f80'),
 ('D', 5, '#20f881'),
 ('L', 6, '#8aa820'),
 ('U', 4, '#3bb9e1'),
 ('L', 6, '#6e2700'),
 ('U', 3, '#4113e3'),
 ('L', 4, '#18af40'),
 ('U', 2, '#38add3'),
 ('L', 6, '#5369b0'),
 ('U', 7, '#7647e3'),
 ('L', 4, '#610f40'),
 ('U', 5

In [6]:
first_dir = data_processed[0][0]
assert first_dir == "R"
current_x = 0
current_y = 0
current = (current_x, current_y)

boundary = [current]

for direction, num, _ in data_processed:

    if direction == "R":
        new = [(current_x, current_y + n) for n in range(1, num + 1)]
    if direction == "L":
        new = [(current_x, current_y - n) for n in range(1, num + 1)]
    if direction == "D":
        new = [(current_x + n, current_y) for n in range(1, num + 1)]
    if direction == "U":
        new = [(current_x - n, current_y) for n in range(1, num + 1)]

    boundary += new
    current = new[-1]
    current_x, current_y = current

assert boundary[0] == boundary[-1]
boundary = boundary[:-1]

In [7]:
len(boundary)

3458

In [8]:
x_max = max(x for x, y in boundary)
x_min = min(x for x, y in boundary)

y_max = max(y for x, y in boundary)
y_min = min(y for x, y in boundary)

num_rows = x_max - x_min + 1
num_cols = y_max - y_min + 1
num_cells = num_rows * num_cols
print(num_rows, num_cols, num_cells)

grid_corners = [(x_min, y_min), (x_min, y_max), (x_max, y_min), (x_max, y_max)]
grid_corners

450 316 142200


[(-319, -55), (-319, 260), (130, -55), (130, 260)]

In [9]:
def get_neighbours_alt(point):
    
    x, y = point
    res = []
    if x > x_min - 1:
        res.append((x - 1, y))
    if x < x_max + 1:
        res.append((x + 1, y))
    if y > y_min - 1:
        res.append((x, y - 1))
    if y < y_max + 1:
        res.append((x, y + 1))

    return res

In [10]:
# We do a flood-fill of the outside. We add an extra row at the top and bottom, and an extra column on the left and right.
start = (x_min - 1, y_min - 1)

boundary_set = set(boundary)

outside_cells_found = set()
cells_to_expand = deque([start]) # Using a deque is much faster than using a list (although we're only appending / popping on the right)

max_num_iter = 1_000_000
num_iter = 0

while cells_to_expand and num_iter < max_num_iter:

    if num_iter % 1000 == 0:
        print(f"Cells found: {len(outside_cells_found)}, to expand: {len(cells_to_expand)}")

    cell_to_expand = cells_to_expand.popleft()
    outside_cells_found.add(cell_to_expand)

    neighbours = get_neighbours_alt(cell_to_expand)
    for neighbour in neighbours:
        if neighbour not in cells_to_expand and neighbour not in outside_cells_found and neighbour not in boundary_set:
            cells_to_expand.append(neighbour)
    
    num_iter += 1

print("done", num_iter)

Cells found: 0, to expand: 1
Cells found: 1000, to expand: 46
Cells found: 2000, to expand: 64
Cells found: 3000, to expand: 78
Cells found: 4000, to expand: 90
Cells found: 5000, to expand: 101
Cells found: 6000, to expand: 111
Cells found: 7000, to expand: 119
Cells found: 8000, to expand: 127
Cells found: 9000, to expand: 135
Cells found: 10000, to expand: 124
Cells found: 11000, to expand: 110
Cells found: 12000, to expand: 113
Cells found: 13000, to expand: 134
Cells found: 14000, to expand: 143


Cells found: 15000, to expand: 156
Cells found: 16000, to expand: 166
Cells found: 17000, to expand: 172
Cells found: 18000, to expand: 165
Cells found: 19000, to expand: 163
Cells found: 20000, to expand: 158
Cells found: 21000, to expand: 136
Cells found: 22000, to expand: 89
Cells found: 23000, to expand: 78
Cells found: 24000, to expand: 101
Cells found: 25000, to expand: 111
Cells found: 26000, to expand: 133
Cells found: 27000, to expand: 131
Cells found: 28000, to expand: 128
Cells found: 29000, to expand: 130
Cells found: 30000, to expand: 137
Cells found: 31000, to expand: 148
Cells found: 32000, to expand: 123
Cells found: 33000, to expand: 118
Cells found: 34000, to expand: 113
Cells found: 35000, to expand: 114
Cells found: 36000, to expand: 115
Cells found: 37000, to expand: 125
Cells found: 38000, to expand: 130
Cells found: 39000, to expand: 122
Cells found: 40000, to expand: 111
Cells found: 41000, to expand: 116
Cells found: 42000, to expand: 125
Cells found: 43000, to

In [11]:
num_cells_inside = 0
for i in range(x_min, x_max):
        for j in range(y_min, y_max):
            cell = (i, j)
            if cell not in boundary_set and cell not in outside_cells_found:
                num_cells_inside += 1

In [12]:
len(boundary)

3458

In [13]:
num_cells_inside

47145

In [14]:
len(boundary) + num_cells_inside

50603

# Part 2

In [15]:
data_corrected = []

for _, _, colour in data_processed:
    num_hex = colour[1:-1]
    num = int(num_hex, 16) # convert hexadecimal to decimal int

    dir = colour[-1]
    if dir == "0":
        dir = "R"
    if dir == "1":
        dir = "D"
    if dir == "2":
        dir = "L"
    if dir == "3":
        dir = "U"

    data_corrected.append((dir, num, None))

In [16]:
data_corrected

[('L', 324704, None),
 ('U', 182348, None),
 ('L', 372065, None),
 ('U', 327173, None),
 ('L', 289159, None),
 ('U', 198815, None),
 ('L', 479065, None),
 ('U', 464499, None),
 ('L', 419559, None),
 ('U', 316292, None),
 ('L', 452969, None),
 ('U', 467255, None),
 ('L', 59085, None),
 ('U', 123476, None),
 ('L', 287874, None),
 ('U', 104522, None),
 ('L', 336848, None),
 ('D', 104522, None),
 ('L', 68716, None),
 ('D', 123476, None),
 ('L', 280932, None),
 ('U', 328817, None),
 ('L', 4059, None),
 ('U', 230280, None),
 ('R', 291018, None),
 ('U', 419130, None),
 ('R', 103452, None),
 ('U', 12730, None),
 ('R', 643044, None),
 ('U', 231701, None),
 ('R', 4136, None),
 ('U', 463774, None),
 ('R', 89418, None),
 ('D', 603507, None),
 ('R', 280824, None),
 ('D', 135048, None),
 ('R', 567938, None),
 ('D', 244638, None),
 ('R', 451184, None),
 ('U', 266558, None),
 ('R', 101108, None),
 ('U', 232157, None),
 ('R', 341659, None),
 ('U', 484478, None),
 ('R', 397556, None),
 ('U', 331352, Non

In [17]:
clockwise = {"U": "R", "R": "D", "D": "L", "L": "U", "U": "R"}

In [18]:
first_dir = data_corrected[0][0]

# vertices are at intersections (not cells)
current_x = 0
current_y = 0
current_vertex = (current_x, current_y)

vertices = [current_vertex]

num_boundary_cells = 0
last_dir = data_processed[-1][0]

num_clockwise_turns = 0
num_counterclockwise_turns = 0

for direction, num, _ in data_corrected:

    if direction == "R":
        new_vertex = (current_x, current_y + num)
    elif direction == "L":
        new_vertex = (current_x, current_y - num)
    elif direction == "D":
        new_vertex = (current_x + num, current_y)
    elif direction == "U":
        new_vertex = (current_x - num, current_y)

    vertices.append(new_vertex)
    current_vertex = new_vertex
    current_x, current_y = current_vertex

    num_boundary_cells += num

    if clockwise[last_dir] == direction:
        num_clockwise_turns += 1
    else:
        num_counterclockwise_turns += 1

    last_dir = direction

assert vertices[0] == vertices[-1]
vertices = vertices[:-1]
num_vertices = len(vertices)

In [19]:
vertices

[(0, 0),
 (0, -324704),
 (-182348, -324704),
 (-182348, -696769),
 (-509521, -696769),
 (-509521, -985928),
 (-708336, -985928),
 (-708336, -1464993),
 (-1172835, -1464993),
 (-1172835, -1884552),
 (-1489127, -1884552),
 (-1489127, -2337521),
 (-1956382, -2337521),
 (-1956382, -2396606),
 (-2079858, -2396606),
 (-2079858, -2684480),
 (-2184380, -2684480),
 (-2184380, -3021328),
 (-2079858, -3021328),
 (-2079858, -3090044),
 (-1956382, -3090044),
 (-1956382, -3370976),
 (-2285199, -3370976),
 (-2285199, -3375035),
 (-2515479, -3375035),
 (-2515479, -3084017),
 (-2934609, -3084017),
 (-2934609, -2980565),
 (-2947339, -2980565),
 (-2947339, -2337521),
 (-3179040, -2337521),
 (-3179040, -2333385),
 (-3642814, -2333385),
 (-3642814, -2243967),
 (-3039307, -2243967),
 (-3039307, -1963143),
 (-2904259, -1963143),
 (-2904259, -1395205),
 (-2659621, -1395205),
 (-2659621, -944021),
 (-2926179, -944021),
 (-2926179, -842913),
 (-3158336, -842913),
 (-3158336, -501254),
 (-3642814, -501254),
 (-3

In [20]:
print(num_counterclockwise_turns)
print(num_clockwise_turns)
print(num_vertices)
print(num_boundary_cells)

333
337
670
172432690


In [21]:
vertices = [vertices[-1]] + vertices + [vertices[0]]

Use Trapezoid formula here
https://en.wikipedia.org/wiki/Shoelace_formula

In [22]:
area = 0
for i in range(1, num_vertices + 1):
    area += (vertices[i][1] + vertices[i + 1][1]) * (vertices[i][0] - vertices[i + 1][0])

area = area / 2

if area < 0:
    # clockwise
    area = abs(area)
    area = area + 3/4 * num_clockwise_turns + 1/4 * num_counterclockwise_turns + (num_boundary_cells - num_vertices) / 2
else:
    # counterclockwise
    area = area + 1/4 * num_clockwise_turns + 3/4 * num_counterclockwise_turns + (num_boundary_cells - num_vertices) / 2

print(area)

96556251590677.0
